# Transformer

In [1]:
import os
import json
import nltk
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import math

In [2]:
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
from google.colab import files

uploaded = files.upload()

Saving transformer_df.csv to transformer_df.csv


In [5]:
df = pd.read_csv("transformer_df.csv")
df = df.drop(columns=['Unnamed: 0'])
df.head()

,Reviews,Summary
0,"['Кроссовки удобные, но быстро порвались.', 'Л...","Кроссовки удобные и лёгкие, но не очень долгов..."
1,"['Телефон быстрый, батарея держит долго.', 'По...",Телефон хороший за свою цену. Камера хорошая д...
2,"['Чайник красивый, но шумный.', 'Пахнет пласти...","Чайник быстро кипятит воду и выглядит стильно,..."
3,"['Качество норм, но после стирки ткань стала ж...","Футболка приятная к телу, но могут быть пробле..."
4,"['Телефон красивый, но корпус маркий – остаютс...",Телефон с хорошей камерой и производительность...


In [6]:
df.head()

,Reviews,Summary
0,"['Кроссовки удобные, но быстро порвались.', 'Л...","Кроссовки удобные и лёгкие, но не очень долгов..."
1,"['Телефон быстрый, батарея держит долго.', 'По...",Телефон хороший за свою цену. Камера хорошая д...
2,"['Чайник красивый, но шумный.', 'Пахнет пласти...","Чайник быстро кипятит воду и выглядит стильно,..."
3,"['Качество норм, но после стирки ткань стала ж...","Футболка приятная к телу, но могут быть пробле..."
4,"['Телефон красивый, но корпус маркий – остаютс...",Телефон с хорошей камерой и производительность...


In [7]:
df["Reviews"] = df["Reviews"].apply(lambda x: " ".join(eval(x)).lower())
df['Reviews'][0]

'кроссовки удобные, но быстро порвались. лёгкие, но скользкие на мокром асфальте. размер в размер, не натирают. через месяц начала отклеиваться подошва. отличные, ношу каждый день!'

## Приведение всех отзывов к одному предложению

In [8]:
df['Reviews'] = df['Reviews'].apply(lambda x: "".join(x.lower()))

In [9]:
df.head()

,Reviews,Summary
0,"кроссовки удобные, но быстро порвались. лёгкие...","Кроссовки удобные и лёгкие, но не очень долгов..."
1,"телефон быстрый, батарея держит долго. после о...",Телефон хороший за свою цену. Камера хорошая д...
2,"чайник красивый, но шумный. пахнет пластиком д...","Чайник быстро кипятит воду и выглядит стильно,..."
3,"качество норм, но после стирки ткань стала жёс...","Футболка приятная к телу, но могут быть пробле..."
4,"телефон красивый, но корпус маркий – остаются ...",Телефон с хорошей камерой и производительность...


In [10]:
from sentence_transformers import SentenceTransformer

In [11]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
df["Reviews"] = df["Reviews"].apply(lambda x: model.encode(x))
df['Reviews'][0]

array([ 2.46777032e-02,  1.16619989e-01,  2.82842815e-01,  5.76891117e-02,
        2.37369552e-01,  8.61389004e-03,  2.29822118e-02,  1.57545030e-01,
       -2.59233236e-01, -1.67967409e-01,  2.33781878e-02,  4.01199430e-01,
        7.99451321e-02,  2.41920069e-01, -1.29370093e-01,  8.36099125e-03,
       -6.85585216e-02, -1.90680437e-02, -1.25378624e-01, -1.47329450e-01,
       -2.06886023e-01,  8.29922035e-02, -5.03693707e-02,  7.00356737e-02,
       -1.06266208e-01,  2.43626848e-01, -2.36048311e-01,  1.84814751e-01,
       -4.38664593e-02, -2.80876726e-01, -1.26140848e-01, -4.23621945e-03,
        1.83917016e-01,  2.44198278e-01,  2.50264555e-01, -1.01010129e-02,
       -2.53838241e-01, -9.15532187e-02, -1.63873509e-01,  1.96698546e-01,
       -1.69608667e-01, -1.42189786e-01, -8.05114284e-02, -3.46762041e-04,
        3.54842603e-01,  3.37279886e-02,  2.89988965e-01,  2.24868774e-01,
       -8.09326675e-03,  1.37084931e-01,  2.72708237e-01,  6.34299591e-02,
       -6.70958534e-02, -

In [13]:
df['Summary'] = df['Summary'].apply(lambda x: model.encode(x))

In [14]:
df.head()

,Reviews,Summary
0,"[0.024677703, 0.11661999, 0.28284281, 0.057689...","[0.12072697, 0.04646999, 0.17353131, -0.010355..."
1,"[-0.14848219, 0.34372818, -0.13571525, 0.01114...","[-0.26656342, 0.4403357, -0.14647679, -0.04963..."
2,"[0.034017913, -0.025875676, 0.04232278, 0.1547...","[0.1088616, 0.0028593147, -0.07166311, 0.20391..."
3,"[0.022648709, 0.08180202, 0.11542283, 0.118454...","[0.06804938, 0.13916674, 0.0005558752, 0.17586..."
4,"[-0.11277293, 0.54605114, -0.17866498, -0.0004...","[-0.19028546, 0.6323152, -0.23953237, -0.00046..."


In [15]:
df.to_csv("bert_transformer.csv")

In [16]:
from google.colab import files

files.download("bert_transformer.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
df['Reviews'][0].shape

(384,)

In [18]:
df['Summary'][0].shape

(384,)

In [19]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


# Реализация нейронной сети

In [20]:
X = np.array(df['Reviews'].tolist())
y = np.array(df['Summary'].tolist())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch

class TransformerHyperModel(HyperModel):
    def build(self, hp):
        inputs = layers.Input(shape=(384,1))

        hidden_units = hp.Int('hidden_units', min_value=128, max_value=1024, step=128)
        attention_heads = hp.Int('attention_heads', min_value=1, max_value=8, step=1)
        dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
        num_transformer_blocks = hp.Int('num_transformer_blocks', min_value=1, max_value=4, step=1)

        x = inputs
        for i in range(num_transformer_blocks):
            attn_output = layers.MultiHeadAttention(
                num_heads=attention_heads,
                key_dim=hidden_units,
                name=f'mha_{i}'
            )(x, x)
            attn_output = layers.Dropout(dropout_rate, name=f'dropout_mha_{i}')(attn_output)
            x = layers.LayerNormalization(name=f'ln_mha_{i}')(x + attn_output)

            ffn_output = layers.Dense(hidden_units, activation='relu', name=f'dense_relu_{i}')(x)
            ffn_output = layers.Dropout(dropout_rate, name=f'dropout_ffn_{i}')(ffn_output)
            x = layers.LayerNormalization(name=f'ln_ffn_{i}')(x + ffn_output)
        x = layers.Flatten()(x)
        x = layers.Dense(384, activation='linear')(x)

        model = Model(inputs, x)

        model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])
        return model


tuner = RandomSearch(
    TransformerHyperModel(),
    objective='val_loss',
    max_trials=5,
    executions_per_trial=1,
    directory='transformer_search',
    project_name='transformer'
)

tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

best_model = tuner.get_best_models(num_models=1)[0]

loss, accuracy = best_model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

perplexity = math.exp(loss)
print(f"Perplexity: {perplexity:.2f}")


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
256               |256               |hidden_units
3                 |3                 |attention_heads
0.4               |0.4               |dropout_rate
3                 |3                 |num_transformer_blocks

Epoch 1/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 168s 9s/step - loss: 163.9858 - mae: 8.7516 - val_loss: 77.4873 - val_mae: 8.6406
Epoch 2/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 213s 10s/step - loss: 17.9966 - mae: 3.6257 - val_loss: 3.3570 - val_mae: 1.5701
Epoch 3/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 159s 9s/step - loss: 3.3698 - mae: 1.4703 - val_loss: 1.3427 - val_mae: 0.9606
Epoch 4/5
 8/17 ━━━━━━━━━━━━━━━━━━━━ 1:18 9s/step - loss: 1.9179 - mae: 1.1028